# Classifying Fronts 2: In code

In [4]:
# imports
from importlib import reload
import os

import numpy as np
import h5py
import xarray
import pandas

from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
import matplotlib.gridspec as gridspec
import seaborn as sns

from skimage import morphology

from wrangler.plotting import cutout
from wrangler.ogcm import llc as wr_llc

from fronts.dbof import utils as dbof_utils
from fronts.dbof import io as dbof_io
from fronts.finding import dev as finding_dev
from fronts.finding import params as finding_params

# DBOF_dev

In [9]:
dbof_dev_file = '../fronts/runs/dbof/dev/llc4320_dbof_dev.json'

# Load up (old files for laptop ease)

In [2]:
cutouts, tbl = finding_dev.load_test_data()

In [5]:
front_params = finding_params.thin_weak_params

# Example

In [3]:
idx = 500
div_sst, sst, sss, Divb2 = finding_dev.parse_idx(cutouts, idx)

## Find the fronts

In [6]:
fronts = finding_dev.algorithms.fronts_from_divb2(Divb2, **front_params)

## Label

In [7]:
flabels = morphology.label(fronts, connectivity=2)

In [9]:
np.unique(flabels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

# Stats

In [16]:
stats = {}

In [17]:
uni_labels = np.unique(flabels)[1:]
stats['FID'] = uni_labels

## Simple stat (<Divb^2>)

In [18]:
avg_Divb2 = []
for ss in uni_labels:
    ipix = flabels == ss
    stat = np.sum(ipix * Divb2) / np.sum(ipix)
    # Save
    avg_Divb2.append(stat)
# 
stats['avg_Divb2'] = avg_Divb2
avg_Divb2

[np.float64(2.1333204661711885e-14),
 np.float64(8.636720066656403e-14),
 np.float64(2.856011945417363e-14),
 np.float64(2.496076959335379e-14),
 np.float64(1.6131827183752875e-14),
 np.float64(1.9997160394594204e-14),
 np.float64(1.748580121842258e-14),
 np.float64(2.3162359135239477e-14),
 np.float64(1.8431564713223905e-14)]

In [19]:
stats['UID'] = [idx]*uni_labels.size

# Length

# Construct a Table

In [20]:
df = pandas.DataFrame(stats)
df

,FID,avg_Divb2,UID
0,1,2.133320e-14,500
1,2,8.636720e-14,500
2,3,2.856012e-14,500
3,4,2.496077e-14,500
4,5,1.613183e-14,500
5,6,1.999716e-14,500
6,7,1.748580e-14,500
7,8,2.316236e-14,500
8,9,1.843156e-14,500


In [24]:
df2 = pandas.DataFrame(stats)
df3 = pandas.concat([df, df2], ignore_index=True)
df3

,FID,avg_Divb2,UID
0,1,2.133320e-14,500
1,2,8.636720e-14,500
2,3,2.856012e-14,500
3,4,2.496077e-14,500
4,5,1.613183e-14,500
5,6,1.999716e-14,500
6,7,1.748580e-14,500
7,8,2.316236e-14,500
8,9,1.843156e-14,500
9,1,2.133320e-14,500
